In [2]:
import pandas as pd
import sqlite3
from pathlib import Path

In [5]:
project_root = Path("..")
raw_csv = project_root / "data" / "raw" / "WA_Fn-UseC_-Telco-Customer-Churn.csv"
db_path = project_root / "data" / "database" / "churn.db"

In [4]:
df = pd.read_csv(raw_csv)

conn = sqlite3.connect(db_path)
df.to_sql("customers_raw", conn, if_exists="replace", index=False)

conn.close()

df.shape

(7043, 21)

In [6]:
import sqlite3
import pandas as pd
from pathlib import Path

project_root = Path("..")
db_path = project_root / "data" / "database" / "churn.db"

conn = sqlite3.connect(db_path)

with open(project_root / "sql" / "01_create_tables.sql", "r") as f:
    conn.executescript(f.read())

pd.read_sql("SELECT COUNT(*) AS rows FROM customers_clean", conn)

,rows
0,7043


### Churn summary from SQL features

Before modeling, we validate that engineered features capture known churn behavior.
This aggregation is computed directly in SQL to reflect how metrics would be
queried in a production analytics environment.

Key observations:
- Customers who churn have substantially shorter tenure
- Churned customers face higher average monthly charges
- The churn rate is meaningfully imbalanced, motivating stratified sampling

In [7]:
query = open(project_root / "sql" / "02_feature_queries.sql").read()
pd.read_sql(query, conn)

,churn,customers,avg_monthly_charges,avg_tenure
0,0,5174,61.27,37.6
1,1,1869,74.44,18.0


In [8]:
with open(project_root / "sql" / "03_validation_checks.sql", "r") as f:
    conn.executescript(f.read())

features = pd.read_sql("SELECT * FROM model_features", conn)
conn.close()

features.head()

,churn,tenure,MonthlyCharges,TotalCharges,is_monthly_contract,has_internet,has_tech_support
0,0,1,29.85,29.85,1,1,0
1,0,34,56.95,1889.5,0,1,0
2,1,2,53.85,108.15,1,1,0
3,0,45,42.30,1840.75,0,1,1
4,1,2,70.70,151.65,1,1,0


In [9]:
conn = sqlite3.connect(db_path)

query = open(project_root / "sql" / "02_feature_queries.sql").read()
summary = pd.read_sql(query, conn)

summary

,churn,customers,avg_tenure,avg_monthly_charges,avg_total_charges,pct_month_to_month,pct_with_tech_support
0,0,5174,37.6,61.27,2549.91,0.429,0.335
1,1,1869,18.0,74.44,1531.80,0.886,0.166
